# 📌 Scraping de Twitter


Este notebook realiza scraping en Twitter/X para recorrer distintas ciudades y varias palabras clave.

Incluye:
- Recolección de texto, fecha, ciudad, keyword y query por combinación ciudad×keyword.
- Manejo de múltiples combinaciones con conteo de tweets por query.
- Construcción de un nombre de archivo por ciudad (para guardar resultados).
- Posibilidad de ejecución asíncrona para acelerar la recolección cuando hay muchas peticiones.


# 📥 Imports y configuración inicial

In [ ]:
from playwright.sync_api import sync_playwright
import nest_asyncio
nest_asyncio.apply()
import asyncio
from playwright.async_api import async_playwright
from datetime import datetime
import pytz

# ⚙️ Definición de funciones auxiliares

# ▶️ Ejecución del scraping

In [ ]:
async def guardar_sesion_twitter_async():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto("https://twitter.com/login")

        print("Inicia sesión manualmente...")

        # Espera que cargue un tweet en el feed
        print("Esperando que cargue el feed...")
        await page.wait_for_selector('div[data-testid="cellInnerDiv"]', timeout=120_000)
        print("Feed detectado. Guardando sesión...")

        await context.storage_state(path="twitter_sesion.json")
        print("Sesión guardada.")
        print("Puedes cerrar el navegador manualmente ahora.")

await guardar_sesion_twitter_async()

Inicia sesión manualmente...
Esperando que cargue el feed...
Feed detectado. Guardando sesión...
Sesión guardada.
Puedes cerrar el navegador manualmente ahora.


# 🕵️ Lógica de scraping principal

In [ ]:
async def scrape_tweets_con_sesion_async(query, max_tweets=200):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(storage_state="twitter_sesion.json")
        page = await context.new_page()

        search_url = f"https://twitter.com/search?q={query}&src=typed_query&f=live"
        await page.goto(search_url)
        print(f"Buscando: {query}")

        try:
            accept_button = await page.wait_for_selector(
                'div[role="button"]:has-text("Aceptar todas las cookies")', timeout=5000
            )
            await accept_button.click()
            print("Cookies aceptadas.")
        except:
            print("No aparecieron cookies.")

        await asyncio.sleep(3)

        tweets = []
        tweets_set = set()
        intentos_sin_nuevos = 0

        while len(tweets) < max_tweets and intentos_sin_nuevos < 5:
            old_count = len(tweets)

            await page.keyboard.press("PageDown")
            await asyncio.sleep(2)

            tweet_containers = await page.query_selector_all('article[data-testid="tweet"]')
            for container in tweet_containers:

                # Expandir tweet si hay "Show more"
                try:
                    show_more_btn = await container.query_selector('button[data-testid="tweet-text-show-more-link"]')
                    if show_more_btn:
                        await show_more_btn.click()
                        await asyncio.sleep(0.3)
                except:
                    pass

                # Texto del tweet
                text_el = await container.query_selector('[data-testid="tweetText"]')
                text = await text_el.inner_text() if text_el else None

                # Fecha del tweet (convertir UTC -> Madrid)
                fecha_local = None
                time_el = await container.query_selector('time')
                if time_el:
                    fecha_str = await time_el.get_attribute('datetime')
                    if fecha_str:
                        try:
                            fecha_utc = datetime.fromisoformat(fecha_str.replace("Z", "+00:00"))
                            tz_madrid = pytz.timezone("Europe/Madrid")
                            fecha_local = fecha_utc.astimezone(tz_madrid).strftime("%Y-%m-%d %H:%M:%S")
                        except Exception as e:
                            fecha_local = fecha_str

                if text and (text, fecha_local) not in tweets_set:
                    tweets.append({
                        "tweet": text,
                        "fecha": fecha_local
                    })
                    tweets_set.add((text, fecha_local))

            if len(tweets) == old_count:
                intentos_sin_nuevos += 1
            else:
                intentos_sin_nuevos = 0

        print(f"Se extrajeron {len(tweets)} tweets.")
        await browser.close()
        return tweets[:max_tweets]

In [ ]:
import json
from datetime import datetime

In [ ]:
ciudades = ['Tenerife', 'Barcelona', 'Madrid', 'Malaga', 'Gran Canaria', 'Seville', 'Valencia', 'Palma de Mallorca']
palabras_clave = [
    "turismo", "viajar", "qué hacer", "actividades", "atracciones",
    "hoteles", "restaurantes", "experiencias", "vacaciones", "monumentos", "lugares"
]

# 💾 Guardado de resultados

In [ ]:
  async def recolectar_tweets_por_combinacion():
      for ciudad in ciudades:
          print(f"Procesando ciudad: {ciudad}")
          tweets_totales = []

          for keyword in palabras_clave:
              await asyncio.sleep(3)
              query = f"{ciudad} {keyword}"
              print(f"Buscando: {query}")

              try:
                  tweets = await scrape_tweets_con_sesion_async(query, max_tweets=500)
              except Exception as e:
                  print(f"Error con '{query}': {e}")
                  tweets = []

              for text in tweets:
                  tweets_totales.append({
                      "tweet": text,
                      "fecha": "2025-08-09T14:32:00.000Z",
                      "ciudad": ciudad,
                      "keyword": keyword,
                      "query": query
                  })

              print(f"{len(tweets)} tweets de '{query}'")

          # Guardar en archivo por ciudad
          timestamp = datetime.now().strftime("%Y%m%d")
          filename = f"{ciudad.replace(' ', '_')}_tweets_{timestamp}.json"
          with open(filename, "w", encoding="utf-8") as f:
              json.dump(tweets_totales, f, ensure_ascii=False, indent=2)

          print(f"Guardado {len(tweets_totales)} tweets en {filename}\n")

In [ ]:
await recolectar_tweets_por_combinacion()

Procesando ciudad: Tenerife
Buscando: Tenerife turismo
Buscando: Tenerife turismo
No aparecieron cookies.
Se extrajeron 504 tweets.
500 tweets de 'Tenerife turismo'
Buscando: Tenerife viajar
Buscando: Tenerife viajar
No aparecieron cookies.
Se extrajeron 473 tweets.
473 tweets de 'Tenerife viajar'
Buscando: Tenerife qué hacer
Buscando: Tenerife qué hacer
No aparecieron cookies.
Se extrajeron 501 tweets.
500 tweets de 'Tenerife qué hacer'
Buscando: Tenerife actividades
Buscando: Tenerife actividades
No aparecieron cookies.
Se extrajeron 180 tweets.
180 tweets de 'Tenerife actividades'
Buscando: Tenerife atracciones
Buscando: Tenerife atracciones
No aparecieron cookies.
Se extrajeron 501 tweets.
500 tweets de 'Tenerife atracciones'
Buscando: Tenerife hoteles
Buscando: Tenerife hoteles
No aparecieron cookies.
Se extrajeron 500 tweets.
500 tweets de 'Tenerife hoteles'
Buscando: Tenerife restaurantes
Buscando: Tenerife restaurantes
No aparecieron cookies.
Se extrajeron 502 tweets.
500 tweet